# Welcome!

Welcome to the first Jupyter notebook in the AI-orchestrated self-driving labs (47332) course. 
We will be using the Jupyter notebooks throughout the course. 

Since you are reading this, you have (probably) have managed to 

1.  install `PumpController` and `Odyssey` packages 
2.  download the notebooks for the course
3.  receive your pumpbot.

You will get some basic information about course exercises in this notebook (consider it a teaser) with significantly 
more to follow next week. 

**The main purpose of this notebook is to verify that you can use the course software and that you can communicate 
with the robot.**

## Exercise 0.1: Verify access to the PumpController

We will now test to see if you have the `PumpController` and can establish a connection with your pumpbot. 

Import the necessary modules for the `PumpController`

In [1]:
from pump_controller import PumpController, get_serial_port, list_serial_ports
from pump_controller import visualize_rgb, visualize_candidates
import time
import numpy as np

If you run the command above without any error messages, it means that you have successfully installed the `PumpController`.

## Exercise 0.2: Establish connection to your pumpbot

Now we initialize the calibratebot. 

We need to figure out which port that the controller is connected to your computer. The `get_serial_port()` function should automatically do this for you, but things have a way of failing when you need them not to. That is why you can call the `list_serial_ports()` function to see all the ports on your computer, and you can simply use the correct port as a string input instead of using the `get_serial_port` function, if the command below does not work. 

The `cell_volume` and `drain_time` properties are already defined to 20 mL and 20 seconds, respectively, but you have the option of changing them here. Notice that a folder called *logs* is created with a file with the current timestamp in it - this is where the colors that you mix on this controller in this session will be stored.


We start with the `config_template.json` file as the `config_file` to begin with, but we will make our own in this tutorial.

Let's runn the initialization command below.

In [ ]:
calibratebot = PumpController(ser_port = get_serial_port(), cell_volume = 20, drain_time = 20, config_file = 'config_files/config_template.json')

If you have successfully initialized the pumpbot, you will see the "Arduino is ready" message. If the "Arduino is ready" message doesn't show up immediately, press the reset button on the Arduino.

Do you see the message? If so, you are now ready to move on to the next step.


## Exercise 0.3: Controlling the pumps of your pumpbot

For calibrating the pumps, we will not be using the test cell, but running the liquid from the respective bottles into a bottle on the scale. So removing the hoses from the test cell and place such that they dispense onto a bottle on the weighing scale.

Before doing anything else, we should purge all the pumps. This means filling all of the tubes with their respective liquids. We do this by using the `purge_pump` function, which takes the pump name ('R', 'G', 'B', 'Y', 'W', 'D') and the time (in seconds) to run the pump, as variables. 

As you can probably guess, the names correspond to the following:

* 'R': red
* 'G': green
* 'B': blue
* 'Y': yellow
* 'W': water
* 'D': drain
   
Do this one-by-one until all tubes are filled with liquid. The drain tubes of course do not need to be purged. 

After purging the red, green, blue, yellow and water pumps, empty the weighing bottle.

In [ ]:
calibratebot.purge_pump('R', 3)

# calibratebot.purge_pump('G', 3)
# calibratebot.purge_pump('B', 3)
# calibratebot.purge_pump('Y', 3)
# calibratebot.purge_pump('W', 3)

# Drain:
# calibratebot.drain()

# Drain for custom time (if required):
# calibratebot.drain(drain_time = 10)

# Flush:
# calibratebot.flush()

## Exercise 0.4: Calibrating the pumps

Now we want to calibrate the pumps. We need to figure out how many mL of water comes out of the pump for every second that we run it. We do this by running the pump for certains amounts of times and weigh the amount of liquid that is output. We can then do a linear regression in Excel to find the slope and the offset for each pump. As each pump is different, these values will also be different. 


The `calibration_test` function cycles through different amounts of time of running the pumps, and waits for 5 seconds. In these 5 seconds, your job is to read the measurement on the weighing scale and note it down in the Excel sheet.

In [ ]:
times = [0.1, 0.2, 0.4, 0.8, 1.0, 1.6, 2.0, 2.5, 3.0, 5.0, 7.0, 8.0]

def calibration_test(pump, times):
    for t in times:
        calibratebot.purge_pump(pump, t)
        print(f"sleeping after {t} on {pump}")
        print("-----------")
        print()
        
        time.sleep(t + 5.0)

Let us run the calibration now! 

Open up the `calibrate_files/pump_calibration_template.xlsx` file and get ready to note down the weights. Optimally, you should do three calibrations for each pump - this results in 6 x 3 = 18 calibrations. If you are pressed for time, just do one calibration per pump and copy the results to the other columns for the same pump. 

Finally, you can take the average `a` (slope) and `b` (intercept) values on the `Main` page of the Excel sheet and write them into a `config.json file`. A template for this is given in `config_files/config_template.json`. Do not change the `pin` values for the different pumps!

In [ ]:
calibration_test('R', times)

# calibration_test('G', times)
# calibration_test('B', times)
# calibration_test('Y', times)
# calibration_test('W', times)
# calibration_test('D', times)

Take these calibration slope and intercept values for each of the pumps, and note them down in the `config.json` file